### Metric test

In [1]:
from DataCompression.msc.Metric import Metric
import numpy as np
from DataCompression.src.decoder_MSE import CNN_AE
from DataCompression.msc.vae_agarap import AE, CNN_AE_network
from DataCompression.src.buffer import Decoder_Buffer
import torch

In [2]:
# load the test dataset
path = f"C:/Users/jabad/Documents/Tuebingen/DataCompression/DataCompression/exp/test_dataset"
db = Decoder_Buffer([3, 210, 160], 128, 1000)
db.load(path)
images = db.images
latents = db.latents

Created a buffer for images of shape torch.Size([1000, 3, 210, 160]) and latents torch.Size([1000, 128])


In [3]:
# reformat the data to fit the decoder method
latent = torch.transpose(torch.unsqueeze(torch.unsqueeze(latents, -1), -1).float(), 2, 1)
x_train = images.float()

In [4]:
# # create the model

# model = CNN_AE(in_dims=latent[0].shape, out_dims=x_train[0].shape)
# criterion = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters())
# epochs = 10

# # train the model
# epoch_losses = []
# for epoch in range(epochs):
#     total_loss = 0
#     # reset the gradients to zero
#     optimizer.zero_grad()

#     # compute reconstructions
#     outputs = model(latent)
#     outputs = torch.squeeze(outputs, 0)

#     # compute reconstruction loss
#     loss = criterion(outputs, x_train)

#     # compute gradients
#     loss.backward()

#     # perform parameter update
#     optimizer.step()

#     # add this images's loss to epoch losses (loss per image normalized)
#     epoch_losses.append(loss.item() / len(images))

#         # display the epoch training loss
#     print("\n epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, epoch_losses[-1]))

In [5]:
# # get the reconstructed images
# outputs = model(latent)

In [6]:
# # run the metric
# metric = Metric(torch.nn.MSELoss())
# accuracy, bpd = metric.measure(x_train, outputs, latent)
# print("MSE Loss:")
# print(accuracy)
# print("bits/image:")
# print(bpd)

In [7]:
# MSE Loss:
# 3619.858154296875
# bits/image:
# 0.576

### Now try for a normal VAE

In [8]:
# train the original vae to get the low dim spaces (this is slow because used single batches)
x_train = images.numpy()
print("Original VAE loss:")
ae = AE()
model = CNN_AE_network(x_train[0].shape, [1, 128, 1])
ae.train_ae(x_train, model, epochs=100)
# get the lower dimensional space it produced
latent = ae.get_latent(x_train)

Original VAE loss:
epoch : 1/10, loss = 5767.670846
epoch : 2/10, loss = 5713.854795
epoch : 3/10, loss = 5663.072440
epoch : 4/10, loss = 5613.801942
epoch : 5/10, loss = 5565.940284
epoch : 6/10, loss = 5519.430361
epoch : 7/10, loss = 5474.241786
epoch : 8/10, loss = 5430.358787
epoch : 9/10, loss = 5387.772857
epoch : 10/10, loss = 5346.480477


In [18]:
# get the reconstructed images
# x_train = torch.unsqueeze(torch.Tensor(x_train), 1)
outputs, _ = model(x_train)

In [19]:
# run the metric
metric = Metric(torch.nn.MSELoss())
latent = np.squeeze(np.asarray(latent), 1)
accuracy, bpd = metric.measure(x_train, outputs, latent)
print("MSE Loss:")
print(accuracy)
print("bits/image:")
print(bpd)

MSE Loss:
5326.24658203125
bits/image:
72.192
